In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing libraries

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.axes
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

## Import data

In [ ]:
train = pd.read_csv("../input/finance-company-loan-data/train_ctrUa4K.csv")
test = pd.read_csv("../input/finance-company-loan-data/test_lAUu6dG.csv")
train_original = train.copy()
test_original = test.copy()

In [ ]:
train.info()

In [ ]:
train["Loan_Status"].value_counts(normalize = True).reset_index()

In [ ]:
train['Loan_Status'].value_counts(normalize = True).plot.bar()

In [ ]:
plt.figure(1)
plt.subplot(221) 
train['Gender'].value_counts(normalize = True).plot.bar(figsize = (20,10), title = 'Gender')
plt.subplot(222)
train['Married'].value_counts(normalize = True).plot.bar(title = 'Married')
plt.subplot(223)
train['Self_Employed'].value_counts(normalize = True).plot.bar(title = 'Self Employed')
plt.subplot(224)
train['Credit_History'].value_counts(normalize = True).plot.bar(title = 'Credit History')



It can be infered from the plots that -
1. 85% applicants in the database are males.
2. 65% applicants are married
3. Around 15% of the applicants are self employed
4. Around 85% of the applicants have repaid their debts

In [ ]:
plt.figure(1)
plt.subplot(131)
train['Education'].value_counts(normalize = True).plot.bar(figsize = (20,5), title = 'Education')
plt.subplot(132)
train['Dependents'].value_counts(normalize = True).plot.bar(title = 'Dependents')
plt.subplot(133)
train['Property_Area'].value_counts(normalize = True).plot.bar(title = 'Property area')

Following can be infered from the plots
1. Around 80% are graduates
2. Most people dont have dependents
3. Most applicants are from semi-urban area

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(train['ApplicantIncome'])
plt.subplot(122)
train['ApplicantIncome'].plot.box(figsize = (16,5))
plt.show()

It can be infered that most of the data in the distribution is inclined towards the left that means data is not normally distributed. We will try to make it normal at later stage.

The boxplot confirms presence of a number of outliers/ extreme values in the database. This can be attributed to the presence of income disparity in the society. This can also be seen from the point of view that we are looking at people with different level of education level

Lets try segregating income by level of education

In [ ]:
train.boxplot(column = 'ApplicantIncome', by = 'Education') 
plt.suptitle("")

We can see that there are a higher number of graduates with very high incomes, which are appearing to be the outliers.

Let’s look at the Coapplicant income distribution

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(train['CoapplicantIncome'])
plt.subplot(122)
train['CoapplicantIncome'].plot.box(figsize =(20,10))

We see a similar distribution as that of the applicant income. Majority of coapplicant’s income ranges from 0 to 5000. We also see a lot of outliers in the coapplicant income and it is not normally distributed.

Let’s look at the distribution of LoanAmount variable.

In [ ]:
plt.figure(1)
plt.subplot(121)
sns.distplot(train['LoanAmount'])
plt.subplot(122)
train['LoanAmount'].plot.box(figsize =(20,10))

In [ ]:
Gender = pd.crosstab(train['Gender'],train['Loan_Status'])
Gender.div(Gender.sum(1).astype(float), axis = 0).plot(kind = "bar",stacked = True, figsize =(4,4))

It can be infered that the approval of loan is not determined by gender.

Now let us see for other variables

In [ ]:
Married = pd.crosstab(train['Married'],train['Loan_Status'])
Education = pd.crosstab(train['Education'],train['Loan_Status'])
Dependents = pd.crosstab(train['Dependents'],train['Loan_Status'])
Self_Employed = pd.crosstab(train['Self_Employed'],train['Loan_Status'])
Married.div(Married.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True, figsize = (4,4))
Education.div(Education.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True, figsize = (4,4))
Dependents.div(Dependents.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True, figsize = (4,4))
Self_Employed.div(Self_Employed.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True, figsize = (4,4))

* Proportion of married applicants is higher for the approved loans.
* Distribution of applicants with 1 or 3+ dependents is similar across both the categories of Loan_Status.
* There is nothing significant we can infer from Self_Employed vs Loan_Status plot.

In [ ]:
bins = []
bins = [0,2500,4000,6000,81000] 
groups = ['Low','Average','High','Very High']
train['Income_bin']=pd.cut(train['ApplicantIncome'], bins, labels = groups)
Income_bin = pd.crosstab(train['Income_bin'], train['Loan_Status'])
Income_bin.div(Income_bin.sum(1).astype(float), axis = 0).plot(kind = "bar", stacked = True,)
plt.xlabel("Applicant Income")
plt.ylabel("Percentage")

It can be inferred that Applicant income does not affect the chances of loan approval which contradicts the hypothesis in which I assumed that if the applicant income is high the chances of loan approval will also be high.

We will analyze the coapplicant income and loan amount variable in similar manner.

In [ ]:
bins = [0,1000,3000,42000] 
groups = ['Low','Average','High']
train['Coapplicant_income_bin']=pd.cut(train['CoapplicantIncome'], bins, labels = groups) 
Coapplicant_Income = pd.crosstab(train['Coapplicant_income_bin'],train['Loan_Status'])
Coapplicant_Income.div(Coapplicant_Income.sum(1).astype(float), axis = 0).plot(kind ="bar", stacked = True, figsize = (4,4))
plt.xlabel("Co-Applicant Income")
plt.ylabel("Percentage")


It shows that if coapplicant’s income is less the chances of loan approval are high. But this does not look right. The possible reason behind this may be that most of the applicants don’t have any coapplicant so the coapplicant income for such applicants is 0 and hence the loan approval is not dependent on it. So we can make a new variable in which we will combine the applicant’s and coapplicant’s income to visualize the combined effect of income on loan approval.

Let us combine the Applicant Income and Coapplicant Income and see the combined effect of Total Income on the Loan_Status.

In [ ]:
bins = [0,2500,4000,81000] 
groups = ['Low','Average','High']
train['Totalincome'] = train['ApplicantIncome'] + train['CoapplicantIncome']
train['Total_income_bin']=pd.cut(train['Totalincome'], bins, labels = groups) 
Total_Income = pd.crosstab(train['Total_income_bin'],train['Loan_Status'])
Total_Income.div(Total_Income.sum(1).astype(float), axis = 0).plot(kind ="bar", stacked = True, figsize = (4,4))
plt.xlabel("Total Income")
plt.ylabel("Percentage")

We can see that Proportion of loans getting approved for applicants having low Total_Income is very less as compared to that of applicants with Average, High and Very High Income.

Let’s visualize the Loan amount variable.

In [ ]:
bins=[0,100,200,700] 
group=['Low','Average','High'] 
train['LoanAmount_bin']=pd.cut(train['LoanAmount'],bins,labels=group)
LoanAmount_bin=pd.crosstab(train['LoanAmount_bin'],train['Loan_Status']) 
LoanAmount_bin.div(LoanAmount_bin.sum(1).astype(float), axis=0).plot(kind="bar", stacked=True, figsize = (4,4)) 
plt.xlabel('LoanAmount') 
P = plt.ylabel('Percentage')

It can be seen that the proportion of approved loans is higher for Low and Average Loan Amount as compared to that of High Loan Amount which supports our hypothesis in which we considered that the chances of loan approval will be high when the loan amount is less.

Let’s drop the bins which we created for the exploration part. We will change the 3+ in dependents variable to 3 to make it a numerical variable.We will also convert the target variable’s categories into 0 and 1 so that we can find its correlation with numerical variables. One more reason to do so is few models like logistic regression takes only numeric values as input. We will replace N with 0 and Y with 1.

In [ ]:
train=train.drop(['Income_bin', 'Coapplicant_income_bin', 'LoanAmount_bin', 'Total_income_bin', 'Totalincome'], axis=1)
train['Dependents'].replace('3+', 3,inplace=True) 
test['Dependents'].replace('3+', 3,inplace=True) 
train['Loan_Status'].replace('N', 0,inplace=True)
train['Loan_Status'].replace('Y', 1,inplace=True)

Now lets look at the correlation between all the numerical variables. We will use the heat map to visualize the correlation. Heatmaps visualize data through variations in coloring. The variables with darker color means their correlation is more.

In [ ]:
matrix = train.corr()
f, ax = plt.subplots(figsize = (9,6))
sns.heatmap(matrix, vmax = .8, cmap = "YlOrBr", square = True)


We see that the most correlated variables are (ApplicantIncome - LoanAmount) and (Credit_History - Loan_Status). LoanAmount is also correlated with CoapplicantIncome.

# Missing value imputation

In [ ]:
train.isnull().sum()

In [ ]:
train['Gender'].fillna(train['Gender'].mode()[0], inplace = True)
train['Married'].fillna(train['Married'].mode()[0], inplace = True)
train['Dependents'].fillna(train['Dependents'].mode()[0], inplace = True)
train['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace = True)
train['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace = True)
train.isnull().sum()

In [ ]:
train['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
train['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)

In [ ]:
test['Gender'].fillna(train['Gender'].mode()[0], inplace=True) 
test['Dependents'].fillna(train['Dependents'].mode()[0], inplace=True) 
test['Self_Employed'].fillna(train['Self_Employed'].mode()[0], inplace=True) 
test['Credit_History'].fillna(train['Credit_History'].mode()[0], inplace=True) 
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True) 
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)


In [ ]:
test['Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].mode()[0], inplace=True)
test['LoanAmount'].fillna(train['LoanAmount'].median(), inplace=True)


In [ ]:
train=train.drop('Loan_ID',axis=1) 
test=test.drop('Loan_ID',axis=1)

In [ ]:
X = train.drop('Loan_Status',1) 
y = train.Loan_Status

In [ ]:
X=pd.get_dummies(X) 
train=pd.get_dummies(train) 
test=pd.get_dummies(test)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_cv, y_train, y_cv = train_test_split(X,y, test_size =0.3)
x_train.head()
x_train.isnull().values.any()

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import accuracy_score
model = LogisticRegression() 
model.fit(x_train, y_train)

In [ ]:
pred_cv = model.predict(x_cv)

In [ ]:
accuracy_score(y_cv,pred_cv)

In [ ]:
pred_test = model.predict(test)

In [ ]:
submission1=pd.read_csv("../input/finance-company-loan-data/sample_submission_49d68Cx.csv", index_col=False)

In [ ]:
submission1['Loan_Status']=pred_test 
submission1['Loan_ID']=test_original['Loan_ID']

In [ ]:
submission1['Loan_Status'].replace(0, 'N',inplace=True) 
submission1['Loan_Status'].replace(1, 'Y',inplace=True)

In [ ]:
pd.DataFrame(submission1, columns=['Loan_ID','Loan_Status']).to_csv('logistic2.csv')